# YOLOv11-seg LGG MRI Tumor Segmentation


In [ ]:
# CELL 1: Install correct & compatible versions (fixes CUDA + YOLOv11)
!pip install -q --no-cache-dir --force-reinstall torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q --no-cache-dir --force-reinstall ultralytics --no-deps
!pip install -q --no-cache-dir opencv-python-headless tqdm kaggle

import torch
from ultralytics import YOLO
import os, cv2, numpy as np, random
from pathlib import Path
from tqdm import tqdm
from IPython.display import FileLink, display, clear_output

print("Installation complete!")
print(f"Ultralytics: {__import__('ultralytics').__version__}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# CELL 2: Upload kaggle.json (drag & drop into left panel)
print("Please DRAG & DROP your kaggle.json file into the file explorer on the left")
print("Then re-run this cell")
clear_output(wait=True)

if not os.path.exists("kaggle.json"):
    print("kaggle.json not found! Please upload it.")
else:
    !mkdir -p ~/.kaggle
    !cp kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json
    print("kaggle.json configured successfully!")

In [ ]:
# CELL 3: Download & extract LGG MRI dataset
!rm -rf lgg-mri-segmentation lgg-mri-segmentation.zip 2>/dev/null || true
!kaggle datasets download -d mateuszbuda/lgg-mri-segmentation --unzip

from pathlib import Path
patients = [p for p in Path("lgg-mri-segmentation").iterdir() if p.is_dir()]
print(f"Dataset ready → {len(patients)} patient folders found")
print("Example:", patients[:3])

In [ ]:
# CELL 4: Convert to YOLO segmentation format
!rm -rf yolo_dataset 2>/dev/null || true

raw = Path("lgg-mri-segmentation")
yolo = Path("yolo_dataset")
for p in ["images/train","images/val","labels/train","labels/val"]:
    (yolo/p).mkdir(parents=True, exist_ok=True)

pairs = []
for patient in raw.iterdir():
    if patient.is_dir():
        for img in patient.glob("*[0-9].tif"):
            mask = patient / f"{img.stem}_mask.tif"
            if mask.exists():
                pairs.append((img, mask))

random.seed(42)
random.shuffle(pairs)
split = int(0.8 * len(pairs))

def convert(img_path, mask_path, folder):
    img = cv2.imread(str(img_path))
    mask = cv2.imread(str(mask_path), 0)
    h, w = img.shape[:2]
    name = f"{img_path.parent.name}_{img_path.name}"
    cv2.imwrite(str(yolo/f"images/{folder}/{name}"), img)
    
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    segs = []
    for cnt in contours:
        if cv2.contourArea(cnt) > 30:
            c = cnt.flatten().astype(float)
            c[0::2] /= w
            c[1::2] /= h
            segs.append(f"0 {' '.join(map(str, c))}")
    if segs:
        with open(yolo/f"labels/{folder}/{name.replace('.tif','.txt')}", "w") as f:
            f.write("\n".join(segs))

for p in tqdm(pairs[:split], desc="Train"): convert(p[0], p[1], "train")
for p in tqdm(pairs[split:], desc="Val")  : convert(p[0], p[1], "val")

with open("data.yaml", "w") as f:
    f.write("path: yolo_dataset\ntrain: images/train\nval: images/val\nnc: 1\nnames: ['tumor']")

print("YOLO dataset ready!")
print(f"Train images: {len(list((yolo/'images/train').glob('*.tif')))}")
print(f"Val images: {len(list((yolo/'images/val').glob('*.tif')))}")

In [ ]:
# CELL 5: Train YOLOv11-seg on GPU (~25–35 min)
model = YOLO("yolov11n-seg.pt")  # auto-downloads first time

model.train(
    data="data.yaml",
    epochs=80,
    imgsz=256,
    batch=32,
    device=0,
    project="runs",
    name="yolov11_lgg",
    exist_ok=True,
    amp=True,
    patience=20,
    workers=4,
    seed=42
)
print("Training completed!")

In [ ]:
# CELL 6: Generate 30-frame professional demo video
best = YOLO("runs/yolov11_lgg/weights/best.pt")
val_imgs = sorted(list(Path("yolo_dataset/images/val").glob("*.tif")))[:30]

frames = []
for p in tqdm(val_imgs, desc="Creating demo video"):
    img = cv2.imread(str(p))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    res = best(p, conf=0.3, verbose=False)[0]
    overlay = img_rgb.copy()
    
    if res.masks is not None:
        for mask in res.masks.data.cpu():
            m = cv2.resize((mask.numpy() > 0.5).astype(np.uint8), (img.shape[1], img.shape[0]))
            overlay = np.where(m[...,None], overlay*0.6 + np.array([0,200,255])*0.4, overlay)
    
    cv2.putText(overlay, "YOLOv11-seg | LGG MRI Tumor Segmentation", (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255,255,255), 2)
    cv2.putText(overlay, "Omid Sakaki", (20, img_rgb.shape[0]-20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (200,255,255), 2)
    
    frames.append(overlay)

out = cv2.VideoWriter("LGG_MRI_YOLOv11_Demo.mp4", cv2.VideoWriter_fourcc(*"mp4v"), 6, (img_rgb.shape[1], img_rgb.shape[0]))
for f in frames:
    out.write(cv2.cvtColor(f, cv2.COLOR_RGB2BGR))
out.release()

display(FileLink("LGG_MRI_YOLOv11_Demo.mp4"))
print("ALL DONE! Click the link above to download your professional demo video!")